# MNIST HiCo Layer 1


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from scipy import stats
from sklearn.metrics import accuracy_score
from keras import backend as K
import cv2

import os
import pickle
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
NUM_TN = 10
SUB_REGION_SCALE = 14
MIN_SUB_REGION_SCALE = 12
MAX_SUB_REGION_SCALE = 16
NUM_HICO_LAYER = 5

#dataset specific parameters
NUM_CLASS = 10

## Data Pre-Processing

In [ ]:
#load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
#Generate cropped train image
X_train_cropped_list = []
coordinate_list = []
scale_list = []
for i in range(NUM_TN):
  X_train_cropped = []
  sub_region_scale = random.randint(MIN_SUB_REGION_SCALE, MAX_SUB_REGION_SCALE)
  x = random.randint(0, X_train.shape[1] - sub_region_scale)
  y = random.randint(0, X_train.shape[1] - sub_region_scale)
  coordinate = (x, y)
  for j in range(X_train.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_row.append(X_train[j][row][column])
      image_cropped.append(image_cropped_row)
    X_train_cropped.append(image_cropped)
  
  X_train_cropped = np.array(X_train_cropped)
  X_train_cropped_list.append(X_train_cropped)
  coordinate_list.append(coordinate)
  scale_list.append(sub_region_scale)

In [ ]:
#Generate cropped test image
X_test_cropped_list = []
for i in range(NUM_TN):
  X_test_cropped = []
  sub_region_scale = scale_list[i]
  x = coordinate_list[i][0]
  y = coordinate_list[i][1]
  for j in range(X_test.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_row.append(X_test[j][row][column])
      image_cropped.append(image_cropped_row)
    X_test_cropped.append(image_cropped)

  X_test_cropped = np.array(X_test_cropped)
  X_test_cropped_list.append(X_test_cropped)

In [ ]:
#rescaling dataset
for i in range(NUM_TN):
  temp = []
  for j in range (X_train.shape[0]):
    temp.append(cv2.resize(X_train_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_train_cropped_list[i] = temp

  temp = []
  for j in range(X_test.shape[0]):
    temp.append(cv2.resize(X_test_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_test_cropped_list[i] = temp

In [ ]:
#normalizing dataset
for i in range(NUM_TN):
  X_train_cropped_list[i] = X_train_cropped_list[i]/255
  X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [ ]:
#transform to vector
for i in range(NUM_TN):
  X_train_cropped_list[i] = X_train_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE))
  X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE))

In [ ]:
#one-hot encoding
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

## HiCo Layer 1

In [ ]:
#build ANN model
ensemble = []
for i in range(NUM_TN):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append((model, i, coordinate_list[i], scale_list[i]))

In [ ]:
#train model
history = []
for i in range(NUM_TN):
  print('Model %d' %i)
  hist = ensemble[i][0].fit(X_train_cropped_list[i], y_train_one_hot, validation_data=(X_test_cropped_list[i], y_test_one_hot), epochs=5, batch_size=128)
  history.append(hist)

Model 0
Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.9628 - accuracy: 0.7016 - val_loss: 0.3646 - val_accuracy: 0.8816
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.3536 - accuracy: 0.8850 - val_loss: 0.2833 - val_accuracy: 0.9052
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 0.2878 - accuracy: 0.9045 - val_loss: 0.2581 - val_accuracy: 0.9104
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 0.2534 - accuracy: 0.9134 - val_loss: 0.2306 - val_accuracy: 0.9188
Epoch 5/5
469/469 [==============================] - 1s 2ms/step - loss: 0.2226 - accuracy: 0.9237 - val_loss: 0.2225 - val_accuracy: 0.9240
Model 1
Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 1.1306 - accuracy: 0.6318 - val_loss: 0.5842 - val_accuracy: 0.7947
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.5700 - accuracy: 0.7957 - val_loss: 0.5195 - val_accuracy: 0.81

In [ ]:
if not os.path.isdir('/content/drive/My Drive/fyp/') :
  os.mkdir('/content/drive/My Drive/fyp/')

with open('/content/drive/My Drive/fyp/MNIST.pickle', 'wb') as f:
    pickle.dump([X_train_cropped_list, y_train_one_hot, X_test_cropped_list, y_test_one_hot, coordinate_list, scale_list], f)

In [ ]:
for i in range(NUM_TN):
  ensemble[i][0].save_weights('/content/drive/My Drive/fyp/MNIST_' + str(i) + '.h5')